# Project: Binomial Lattice Models of Equity Share Price

## Learning Objectives
In this project students will rigoursly test the effectivness of the binomial lattice model as a predictive approach for short-term projection of equity share price. 

* __Objective__: Quantify the binomial lattice model prediction rate by comparing historical data for different periods and different firms with lattice model predictions.

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our example problem.

### Packages
Fill me in.

### Types
Fill me in.

### Functions
Fill me in.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-132/module-1`


## Prerequisite: Load the Binomial Lattice Model Parameters and Historical Dataset
Fill me in

In [2]:
parameters = loadmodelparametersfile();

Fill me in

In [3]:
years = ["Year-1", "Year-2", "Year-3", "Year-4", "Year-5"];
number_of_firms = nrow(parameters);
dataset = Dict{Int64,DataFrame}();
for i ∈ 1:number_of_firms
    firm_index = parameters[i,:firm];
    for year ∈ years
        tmp_data = loaddatafile(firm=firm_index, year=year);
        
        if (haskey(dataset,firm_index) == false)
            dataset[firm_index] = tmp_data;
        else
            append!(dataset[firm_index], tmp_data)
        end
    end
end

### Clean the dataset: missing data
Not all the firms in the `dataset` have the maximum number of trading days, i.e., some firms are missing information for various reasons; perhaps they were acquired, merged, or were delisted, etc. We will exclude these firms from the `dataset`.

#### Implementation
First, we specify the `max_number_of_records` variable, which holds the number of trading records we expect for each firm, in this case, `1256`. Then, we iterate through all the firms in the `dataset` using a `for` loop. We remove those firms from the `dataset` whose row count (calculated by calling the `nrow(...)` function) is not equal to the `max_number_of_records`; a firm is removed from the `dataset` using the [delete! function](https://docs.julialang.org/en/v1/base/collections/#Base.delete!):

In [4]:
max_number_of_records = 1256
for i ∈ 1:number_of_firms
    firm_index = parameters[i,:firm];
    data = dataset[firm_index];
    
    if (nrow(data) != max_number_of_records)
        delete!(dataset, firm_index);
    end
end

Fill me in

In [5]:
list_of_firm_ids = keys(dataset) |> collect |> sort;
number_of_firms = length(list_of_firm_ids);

## Objective: Quantify the Binomial Lattice Model Prediction Rate
Fill me in

In [6]:
T = 21;

Fill me in

In [7]:
date_range_array = Array{StepRange{Int64, Int64},1}();
start_index = 1;
for i ∈ 1:T:(max_number_of_records - T)
    range(i,stop=(i+T-1),step=1) |> (x-> push!(date_range_array,x))
end
number_of_date_ranges = length(date_range_array);

Fill me in

In [8]:
z_score = [1.0,1.96,2.576]
successful_simulation_dict = Dict{Int64, Array{Array{Float64,2},1}}()
failed_simulation_dict = Dict{Int64, Array{Array{Float64,2},1}}()
for i ∈ 1:number_of_firms
    firm_id = list_of_firm_ids[i];
    
    # parameters -
    ū = parameters[parameters.firm .== firm_id, :up] |> first;
    d̄ = parameters[parameters.firm .== firm_id, :down] |> first;
    p̄ = parameters[parameters.firm .== firm_id, :probability] |> first;
    
    # data for this firm
    data = dataset[firm_id];
    
    # process each data range, for firm_id
    for j ∈ 1:number_of_date_ranges
        date_range = date_range_array[j]
        start_index = first(date_range)
        
        # set the starting price
        Sₒ = data[start_index, :volume_weighted_average_price];
        
        # build the model, populate with price estimates 
        model = build(MyBinomialEquityPriceTree, (
            u = ū, d = d̄, p = p̄)) |> (x-> populate(x, Sₒ, T));
        
        
        simulation_data_array = Array{Float64,2}(undef, T, 10);
        for k ∈ 0:(T-1)
            simulation_data_array[k+1,1] = k+start_index
            simulation_data_array[k+1,2] = data[start_index+k,:volume_weighted_average_price]
            simulation_data_array[k+1,3] = 𝔼(model,level=k);
            simulation_data_array[k+1,4] = 𝕍(model,level=k) |> sqrt;

            # compute an upper and lower bound -
            simulation_data_array[k+1,5] = simulation_data_array[k+1,3] .- z_score[1]*simulation_data_array[k+1,4]
            simulation_data_array[k+1,6] = simulation_data_array[k+1,3] .+ z_score[1]*simulation_data_array[k+1,4]
            
            # compute an upper and lower bound -
            simulation_data_array[k+1,7] = simulation_data_array[k+1,3] .- z_score[2]*simulation_data_array[k+1,4]
            simulation_data_array[k+1,8] = simulation_data_array[k+1,3] .+ z_score[2]*simulation_data_array[k+1,4]
            
            # compute an upper and lower bound -
            simulation_data_array[k+1,9] = simulation_data_array[k+1,3] .-  z_score[3]*simulation_data_array[k+1,4]
            simulation_data_array[k+1,10] = simulation_data_array[k+1,3] .+ z_score[3]*simulation_data_array[k+1,4]
        end
        
        # was this a good prediction?
        L = simulation_data_array[:,9]
        U = simulation_data_array[:,10]
   
        # check the bounds: if below L, or above U then simulation failed
        prediction_ok = true;
        for k ∈ 1:T
            if (data[start_index+k-1,:volume_weighted_average_price] < L[k]) || (data[start_index+k-1,:volume_weighted_average_price] > U[k])
                prediction_ok = false;
                break;
            end
        end
        
        key = firm_id
        tmp = Array{Array{Float64,2},1}();
        if (prediction_ok == true)
            
            if (haskey(successful_simulation_dict, key) == false)
                push!(tmp,simulation_data_array) |> (x-> successful_simulation_dict[key] = x);
            else
                
            end
        else
            
            if (haskey(failed_simulation_dict, key) == false)
                push!(tmp,simulation_data_array) |> (x-> failed_simulation_dict[key] = x);
            else
                push!(failed_simulation_dict[key],simulation_data_array);
            end
        end
    end
end

In [12]:
length(failed_simulation_dict[56])

14

In [10]:
successful_simulation_dict[56][1]

21×10 Matrix{Float64}:
  1.0  750.062  750.062   0.0     …  750.062  750.062  750.062  750.062
  2.0  760.38   750.868   8.0008     735.187  766.55   730.258  771.478
  3.0  773.222  751.676  11.3273     729.474  773.877  722.497  780.855
  4.0  770.52   752.484  13.8884     725.263  779.706  716.708  788.261
  5.0  769.196  753.294  16.0547     721.827  784.761  711.937  794.65
  6.0  767.724  754.104  17.9695  …  718.884  789.324  707.815  800.393
  7.0  766.687  754.915  19.7063     716.291  793.539  704.151  805.678
  8.0  784.26   755.727  21.3087     713.962  797.492  700.836  810.618
  9.0  782.472  756.54   22.8051     711.842  801.238  697.794  815.286
 10.0  785.988  757.353  24.2152     709.892  804.815  694.975  819.732
 11.0  785.866  758.168  25.5532  …  708.084  808.252  692.343  823.993
 12.0  792.188  758.983  26.8301     706.396  811.57   689.869  828.098
 13.0  782.702  759.8    28.054      704.814  814.785  687.532  832.067
 14.0  783.395  760.617  29.2318     703.3